In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline
from SimpleCV import *
disp = Display(displaytype='notebook')
import seaborn as sns
import glob
import json

In [ ]:
from autopump import autopump

In [ ]:
AP = autopump.AutoPump()

In [ ]:
def returnCalibrationCurve(imgdir):
    #returns calibration curve from an image directory.  Assumes imgs are named XXml.jpg where XX is the volumne in ml
    imgs = glob.glob(imgdir + '/*.*')
    vols = list()
    out = list()
    for imgfile in imgs:
#        img = Image(imgfile)
#        AP.img = img
        out.append(AP.processVision(imgfile))
        vols.append(imgfile.split('ml')[0].split('/')[-1])
    out = np.array(out)
    return [out,np.array(vols).astype('int64')]

In [ ]:
returnCalibrationCurve('/Users/ryan/Code/DataDumps/AutoPump/camtesting/vision2/cal/calcurve3/run2/')

In [ ]:
out1,vols1 = returnCalibrationCurve('/Users/ryan/Code/DataDumps/AutoPump/camtesting/vision2/cal/calcurve3/run1/')
out2,vols2 = returnCalibrationCurve('/Users/ryan/Code/DataDumps/AutoPump/camtesting/vision2/cal/calcurve3/run2/')
#out3,vols3 = returnCalibrationCurve('GaugeRR/run3/')

In [ ]:
fit = np.polyfit(out1[:,0], np.array(vols1),1)

In [ ]:
fit

In [ ]:
AP.saveConfig()

In [ ]:
#del tmp['img']
tmp['breathCounter'] = tmp['breathCounter'].value

In [ ]:
tmp.keys()

In [ ]:
tmp

In [ ]:
tmp['img'] = []

In [ ]:
tmp = AP.__dict__

In [ ]:
fit

In [ ]:
np.polyval(fit,800)

In [ ]:
fit

In [ ]:
fity = np.linspace(1300,100)
fitx = list()
for fy in fity:
    fitx.append(np.polyval(fit,fy))

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(fitx,fity,'--',label='fit')
plt.plot(vols1,out1[:,0],'d',markersize=8,label='run1')
plt.plot(vols2,out2[:,0],'d',markersize=8,label='run2')
#plt.plot(vols3,out3[:,0],'d',markersize=8,label='run3',)
plt.legend()
#plt.legend({'fit','run1','run2','run3'})
plt.axis([200,550,0,1600])
plt.xlabel('ml pumped')
plt.ylabel('y position (pixels)')
plt.show()

In [ ]:
img = Image('115ml.jpg')
img = img.rotate270()
cylinder = img.crop([100,480,400,2000])
ballimg = cylinder.hueDistance(color=(130,110,55)).invert()
channels = cylinder.splitChannels()
#blobs = red_channel.findBlobs(threshold)

In [ ]:
imgdir = '/Users/ryan/Code/DataDumps/AutoPump/camtesting/vision2/cal/calcurve2/run3/'
imgs = glob.glob(imgdir + '/*.*')
outvals = list()
thresh = -0.005
for i in imgs:
    img = Image(i)
    img = img.rotate270()

    cylinder = img.crop([250,950,300,2000]) #extents for image 
    bluechan = cylinder.getNumpy()[:,:,0]
    bluechan = bluechan.mean(axis=1)
    smoothb = smooth(bluechan,window_len=50)
    try:
        ballheight = np.nonzero(np.diff(diff(smoothb)) < thresh)[0][0] #get first y value that exceeds thrshold 
    except:
        logging.warning('Machine vision failure: ball not detected')
        ballheight = -1
    outvals.append(ballheight)


In [ ]:
outvals

In [ ]:
img = Image('/Users/ryan/Code/DataDumps/AutoPump/camtesting/vision2/cal/calcurve3/run1/0ml.jpg')
#img = Image('../../../DataDumps/AutoPump/camtesting/pingpong/GaugeRR/run1/115ml.jpg')
img = img.rotate270()

cylinder = img.crop([250,950,300,2000]) #extents for image 
bluechan = cylinder.getNumpy()[:,:,0]
bluechan = bluechan.mean(axis=1)
smoothb = smooth(bluechan,window_len=50)
try:
    ballheight = np.nonzero(np.diff(smoothb) < -0.15)[0][0] #get first y value that exceeds thrshold 
except:
    logging.warning('Machine vision failure: ball not detected')
    ballheight = -1

#mls = ballheight * self.heightToMl

In [ ]:
cylinder.save(disp)

In [ ]:
plt.plot(smoothb)

In [ ]:
plt.plot(diff(diff(smoothb)))

In [ ]:
plt.plot(diff(smoothb))
plt.hlines(-0.25,0,1800)
plt.axis([0, 2000, -0.6, 0.8])

In [ ]:
ballheight

In [ ]:
cylinder.save(disp)

In [ ]:
r = cylinder.getNumpy()[:,:,2]
r = r.mean(axis=1)
g = cylinder.getNumpy()[:,:,1]
g = g.mean(axis=1)
b = cylinder.getNumpy()[:,:,0]
b = b.mean(axis=1)



In [ ]:
plt.plot(r,'r')
plt.plot(g,'g')
plt.plot(b,'b')

In [ ]:
def smooth(x,window_len=11,window='hanning'):
        if x.ndim != 1:
                raise ValueError, "smooth only accepts 1 dimension arrays."
        if x.size < window_len:
                raise ValueError, "Input vector needs to be bigger than window size."
        if window_len<3:
                return x
        if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
                raise ValueError, "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
        s=numpy.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
        if window == 'flat': #moving average
                w=numpy.ones(window_len,'d')
        else:  
                w=eval('numpy.'+window+'(window_len)')
        y=numpy.convolve(w/w.sum(),s,mode='same')
        return y[window_len:-window_len+1]

In [ ]:
smoothb = smooth(b,window_len=50)

In [ ]:
plt.plot(smoothb)

In [ ]:
#plt.plot(tmp)
plt.plot(np.diff(smoothb))

In [ ]:
np.nonzero(np.diff(smoothb) < -0.15)[0][0]

In [ ]:
cylinder.save(disp)